# ReAct

ReAct is one of the earliest _thinking_ prompting techniques where the LLM was asked to breakdown it's task into multiple turns and incorporate external tools in it's process before responding. The hope _(and finding)_ is that it would move away from simple recall to a more complex chain of thought yielding more appropriate answers.

The most basic `ReAct` form is the one mentioned in the paper that introduced it : [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/pdf/2210.03629). See also the companion [react-lm.github.io](https://react-lm.github.io/) site.

One of the confusing things about ReAct is that the prompts and examplars used vary widely. However, they all share a similar high-level structure: prompt engineering + COT exemplar to think along ReActy lines.

 - ReAct specific prompt
   - Clarity of the ReAct instruction in the role
   - Tool specification can vary: either fully natural language or mix in JSON notation for arguments
   - Reduce hallucinations on failure: _Exemplar allowing the LLM to fail with a `I cannot answer this` instead of forcing it to hallucinate an answer_
 - General Prompt
   - Re-iterate important instructions at the end
   - Re-iterate function calling semantic rules to follow including JSON formats

I find it useful to break the prompt down into sections so I have a common structure to study different examples. Otherwise, we end up with a wall-of-diffs that does not guide us toward tweaking our own prompts.

## Experimentation results

I started coding the ReAct loop based on the previously developed [tool infrastructure](./Py_mod_llm_agentic_toolcalling_devel.ipynb). While the conclusions were reached much further in this notebook, I will present their summary here for the impatient.

### React activity

![](../img/react_llm_next_steps.png)

### ReAct overlayed on the chat protocol

![](../img/react_chat_mm.png)

----

![](../img/react_highlevel_protocol.png)


## Setup notebook py environment


In [1]:
# Allow Colab or VSCore/Normal Jupyter environments
import os
from pathlib import Path

# The default relative path when running the notebook from a cloned repo.
LIB_PATH = Path("../lib")

if 'google.colab' in str(get_ipython()):
    print("👉 Setting up for Colab")
    # This will create a py-llm dir at the same level as this notebook
    # Refer to the lib in there using `./py-llm/lib` as opposed to the 
    # relative `../lib` when we are running straight from the py-llm/nbs 
    # directory in VScode.
    if not os.path.isdir("./py-llm"):
        print("Cloning git repo into ./py-llm")
        !git clone https://github.com/vamsi-juvvi/py-llm.git
        LIB_PATH = Path("./py-llm/lib")
    else:
        print("./py-llm exists. Not cloning") 

In [2]:
# Append to sys.path directly
# Make sure to `str(Path)`
# - The resolve() converts relative to absolute. 
# - If you use ~ for HOME, use `Path.expand_user()`
import sys
import logging

sys.path.append(str(LIB_PATH.resolve()))

from py_llm.util import jupyter_util
from py_llm.util.jupyter_util import TextAlign
from py_llm.util.jupyter_util import DisplayHTML as DH
from py_llm.util.jupyter_util import DisplayMarkdown as DM
from py_llm.util.jupyter_util import ColabEnv

from py_llm.llm import openai_util as oai
from py_llm.llm.tools import Tool, ToolCollection

# Init jupyter env
jupyter_util.setup_logging(logging.WARNING)

In [3]:
# Uncomment for use in Colab or when the package is missing
#!pip install -qy openai

In [4]:
import openai

# If you want to log OpenAI's python library itself, also set the log level for this
# normally, limit this to warning/error and keep your own logging at debug levels.
# If this doesn't work right away, restart the kernel after changing the log-level
os.environ["OPENAI_LOG"]="error"

# Finally ensure you have the OpenAI key.
openai.api_key = ColabEnv.colab_keyval_or_env("OPENAI_API_KEY")
assert(openai.api_key)



## Build on top of existing tool-calling agent

In [5]:
#import py_llm.llm.agentic.tool_calling as tooled_agent



## ReAct system prompt structure and template

The examples you see online are huge blocks of text. There is a structure to the pompt but it is obscured by all the text. I want to make the structure explicit so it is easier to reason about it and maybe tune it in sections.

I will use the following template as the initial breakdown (based on prompt at https://docs.llamaindex.ai/en/stable/examples/agent/react_agent/) and tune it as I go along. The goal is to end up with a template and a `builder` to craft the ReAct system prompt.

In [6]:
from string import Template

react_system_prompt_template = Template("""
                                        
$YOUR_ROLE_AS_REACT_SECTION
                                        
$REACT_TOOLS_SECTION

$REACT_LOOP_EXEMPLARS_SECTION
                                        
$REACT_LOOP_ADDITIONAL_RULES
                                        
$REACT_INTRODUCE_CONVERSATION_SECTION
""".strip())

I will use the following dictionary to hold the substitution variables _(Template terminology: a dictionary keyed by the variable names will will eventually replace the variables in the template)_. Some of the values are themselves _Templates_ if their semantics allows for further deconstruction.

In [7]:
react_sys_prompt_template_args = {}

react_sys_prompt_template_args["YOUR_ROLE_AS_REACT_SECTION"] = """
You are designed to help with a variety of tasks, from answering questions \
to providing summaries to other types of analyses.""".strip()

# + TOOLS
react_sys_prompt_template_args["REACT_TOOLS_SECTION"] = Template("""
## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.

You have access to the following tools:
$TOOLS
""".strip())

# + TOOL_NAMES_CSV
# + REACT_CONCLUSION_WITH_SUCCESS_EXEMPLAR
#     Thought: I can answer without using any more tools.
#     Answer: [your answer here]
#
# + REACT_CONCLUSION_WITH_FAILURE_EXEMPLAR
#     Thought: I cannot answer the question with the provided tools.
#     Answer: Sorry, I cannot answer your query.
react_sys_prompt_template_args["REACT_LOOP_EXEMPLARS_SECTION"] = Template("""
## Output Format
To answer the question, please use the following format.

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of $TOOL_NAMES_CSV) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond with an observation in the following format:

```
Observation: tool response
```

You should keep repeating the above format until you have enough information
to answer the question without using any more tools. At that point, you MUST respond
in the one of the following two formats:

```
$REACT_CONCLUSION_WITH_SUCCESS_EXEMPLAR
```

```
$REACT_CONCLUSION_WITH_FAILURE_EXEMPLAR
```
                                                                          
Please Pay attention to the following instructions:
  - You MUST obey the function signature of each tool. Do NOT pass in no arguments if the function expects arguments.
""".strip())

# In the LlamaIndex example, this was done as an enhancement step when 
# refining the prompt. Retain it as the LLAMA_REACT_LOOP_ADDITIONAL_RULES 
# constant (if you want to try it out) but default it to "".
react_sys_prompt_template_args["REACT_LOOP_ADDITIONAL_RULES"] = ""

react_sys_prompt_template_args["REACT_INTRODUCE_CONVERSATION_SECTION"] = """
## Current Conversation
Below is the current conversation consisting of interleaving human and assistant messages.
""".strip()

#--------------------------------------------------------------------------
# Extra constants
#--------------------------------------------------------------------------
# Possible values to try for additional rules. Taken from the LlamaIndex 
# examples
LLAMA_REACT_LOOP_ADDITIONAL_RULES = """
## Additional Rules
- The answer MUST contain a sequence of bullet points that explain how you arrived at the answer. This can include aspects of the previous conversation history.        
""".strip()


In [8]:
import copy

def are_all_vars_resolved(tmpl: Template) -> bool :
        try:
            tmpl.substitute({})
            return True
        except ValueError:
            return False        

class ReactSysPromptBuilder:    

    def __init__(self):        
        # the keys of subst_args will be resolved incrementally. So use a 
        # deep copy to leave template args intact.
        self.subst_args = copy.deepcopy(react_sys_prompt_template_args)
        self.tmpl       = react_system_prompt_template

    #-------------------------------------    
    def build_safe(self) -> str:
         """Does not fail even if variables are unresolved"""
         # low level resolve in a copy
         resolved_args = copy.deepcopy(self.subst_args)
         for k,v in resolved_args.items():
              if isinstance(v, Template):
                   resolved_args[k] = v.safe_substitute({})

         return self.tmpl.safe_substitute(resolved_args)

    #-------------------------------------
    def override_role(self, role_arg: str | None):
         """
         Call if you want a different role than the default
         ReAct role.
         """
         ROLE_SECTION_KEY = "YOUR_ROLE_AS_REACT_SECTION"
         if role_arg:
            self.subst_args[ROLE_SECTION_KEY] = role_arg        

    #-------------------------------------
    def init_tools_tmpl(self, tools_arg:str):
        TOOLS_SECTION_KEY="REACT_TOOLS_SECTION"
        TOOLS_CHILD_KEY="TOOLS"          

        # Update the EXEMPLARSSECTION_KEY template
        self._do_update_tmpl_arg(
             TOOLS_SECTION_KEY,
             {
                TOOLS_CHILD_KEY : tools_arg if tools_arg else "TOOLS is NOT SPECIFIED"
             })            
    
    #-------------------------------------
    def init_exemplars_tmpl(self, 
                            tool_names_csv : str,
                            success_example:str | None, 
                            cannot_answer_example: str | None):
        """
        Must be called to set the tool_names_csv as it has no meaningful default.
        
        The success_example and cannot_answer_examples can be overridden 
        but have default values.
        """

        EXEMPLARSSECTION_KEY="REACT_LOOP_EXEMPLARS_SECTION"

        CHILD_KEY_TOOL_NAMES_CSV = "TOOL_NAMES_CSV"
        CHILD_KEY_SUCCESS        = "REACT_CONCLUSION_WITH_SUCCESS_EXEMPLAR"
        CHILD_KEY_CANNOT_ANSWER  = "REACT_CONCLUSION_WITH_FAILURE_EXEMPLAR"

        # Defaults        
        DEFAULT_SUCCESS_EXAMPLE = """
Thought: I can answer without using any more tools.
Answer: [your answer here]
            """.strip()
             
        DEFAULT_CANNOT_ANSWER_EXAMPLE = """
Thought: I cannot answer the question with the provided tools.
Answer: Sorry, I cannot answer your query.
             """.strip()        

        # Update the EXEMPLARSSECTION_KEY template
        self._do_update_tmpl_arg(
             EXEMPLARSSECTION_KEY,
             {
                CHILD_KEY_TOOL_NAMES_CSV : tool_names_csv if tool_names_csv else "Tool names NOT SPECIFIED",
                CHILD_KEY_SUCCESS        : success_example if success_example else DEFAULT_SUCCESS_EXAMPLE,
                CHILD_KEY_CANNOT_ANSWER  : cannot_answer_example if cannot_answer_example else DEFAULT_CANNOT_ANSWER_EXAMPLE,
            })                
        
    #-------------------------------------
    def init_additional_rules_tmpl(self, 
                            additional_rules : str | None = None):
        """
        You don't always want additional rules. 
        Defaults to empty "". 

        Call with REACT_LOOP_ADDITIONAL_RULES constant if you want to try 
        the 'conclude with reasons` version that LlamaIndex tried in their refined
        version of the prompt.
        """        
        ADDITIONAL_RULES_SECTION_KEY="REACT_LOOP_ADDITIONAL_RULES"                
        self._do_update_string_arg(
            ADDITIONAL_RULES_SECTION_KEY,
            additional_rules
        )         
    
    #-------------------------------------
    def _do_update_tmpl_arg(self, key:str, subst_dict:dict):
        """
        Updates self.subst_args[key] 's template value with the supplied
        substitutions. The updated value will remain a Template when all
        values are fully resolved.
        
        This is done safely with no exceptions which means 
        the resulting template can still have unresolved variables. 
        """
        assert(isinstance( self.subst_args[key], Template))

        self.subst_args[key] = Template(
                 self.subst_args[key].safe_substitute(subst_dict)
        )
    
    def _do_update_string_arg(self, key:str, subst_str:str|None=None):
        """
        Replaces self.subst_args[key] 's string value with the supplied
        string. 

        If None is suppplied, then this is cleared out (set to "")
        """
        assert(isinstance( self.subst_args[key], str))

        self.subst_args[key] = subst_str if subst_str else ""


----

# ReAct - LlamaIndex example

See https://docs.llamaindex.ai/en/stable/examples/agent/react_agent/

Recreate this from the above system prompt. This will be straightforward since the template was deconstructed from this prompt.

Note:
 - They keep the `REACT_LOOP_ADDITIONAL_RULES` section blank first time around
 - Setting it to `LLAMA_REACT_LOOP_ADDITIONAL_RULES` apparently leads to it describing the actions it took.

**Caveat**: For some models that support tool-calling natively, they say that you might need an option to map `Observation` to `role="tool"` used in typical tool-call responses _(for OpenAI atleast)_. They that this is handled via 

```python
from llama_index.core.agent import ReActChatFormatter
from llama_index.core.llms import MessageRole

agent = ReActAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    react_chat_formatter=ReActChatFormatter.from_defaults(
        observation_role=MessageRole.TOOL
    ),
    verbose=True,
)
```

This should be straigtforward to do for me as well as specifying a `"role" : blah` is standard procedure.

When exercisig the LlamaIndex simple use-case, I had to enhance my tool framework (See [Py_mod_llm_tools_devel.ipynb](./Py_mod_llm_tools_devel.ipynb) and [Py_mod_llm_toolcalling_devel.ipynb](./Py_mod_llm_toolcalling_devel.ipynb))
 - ✔️ Already allows tools of the form `fn(PydanticModel)`
 - ➕ Now allows regular python methods of the form `fn(a, b, c)`. Tested for primitive types and will fix limitations as I come across them.

In [9]:
# https://docs.llamaindex.ai/en/stable/examples/agent/react_agent/
# uses the following tools.
def multiply(a:int, b:int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def add(a:int, b:int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

# Add these to a tool-collection
react_1_tc = ToolCollection([multiply, add])

# sanity checks
if 0:
    # test out the schemas needed for ReAct: get_inprompt_schemas()
    # Note that for regular tool calling, we use the get_schemas() method.
    for ts in react_1_tc.get_inprompt_schemas(mapper = lambda ps: str(ps)):
        DM.hr()
        DM.code(ts)

    print(react_1_tc.get_tool_names())
    DM.hr()

    # test direct execution
    import json
    print(
        react_1_tc.exec_tool("add", json.dumps({"a":3, "b":8}))
        )

In [10]:
builder = ReactSysPromptBuilder()

builder.init_tools_tmpl(
    tools_arg = "\n" + "\n\n".join(
        react_1_tc.get_inprompt_schemas(mapper = lambda ps: str(ps))
    )
)

builder.init_exemplars_tmpl(
    tool_names_csv = ", ".join(react_1_tc.get_tool_names()),
    success_example = None,
    cannot_answer_example = None)

builder.init_additional_rules_tmpl(
    additional_rules=None)

print(builder.build_safe())

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.

You have access to the following tools:

Tool Name : multiply
Tool Description : Multiply two integers and returns the result integer
Tool Args: {"properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"], "title": "multiply_args", "type": "object", "additionalProperties": false}

Tool Name : add
Tool Description : Add two integers and returns the result integer
Tool Args: {"properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"], "title": "add_args", "type": "object", "additionalProp

In [11]:
import re
import logging

class ReactAssistantResponse:
    PATTERN_TH        = re.compile(r"^(Thought|Action|Action Input|Answer)\s*:\s*(.*?)$", re.MULTILINE)
    PATTERN_FUNC_NAME = re.compile(r"^\s*(?:function[s]?\.)?(.*)$")

    def __init__(self, assistant_response:str):
        self.thought      = None
        self.action       = None
        self.action_input = None
        self.answer       = None

        #-- Parse -----------------        
        match_list = self.PATTERN_TH.findall(assistant_response)

        if match_list or len(match_list) > 0:
            d = {}
            for m in match_list:
                key = m[0]
                val = m[1]
                logging.debug(f"Extracted [{key} = {val}] pair")
                d[key] = val
            self._init_kvps(d)
        else:
            logging.debug("Could not extract any exected React semantic sections from assistant response\n{assistant_response}")
    
    def __str__(self):
        return f"""
Thought     : {self.thought}
Action      : {self.action}
Action Input: {self.action_input}""".strip()

    def _init_kvps(self, d):
        for k,v in d.items():
            match k.lower():
                case "thought":
                    self.thought = v.strip()
                case "action":
                    # These seem to sometimes comes in as `function.my_action`
                    fn_match = self.PATTERN_FUNC_NAME.match(v)
                    if fn_match:
                        self.action = fn_match.group(1)
                        logging.debug(f"Got Action = {self.action}")
                    else:
                        logging.error(f"Unable to get function name from Action=\"{v}\"")
                case "action input":
                    self.action_input = v
                    logging.debug(f"Got Action Input = {v}")
                case "answer":
                    self.answer = v
                    logging.debug(f"Got Answer = {v}")
                case _:
                    logging.warning(f"Unknown Key={k} with Value={v}")

In [12]:
class ReactObservation:    
    def format_observation(content:str):
        # Note that our system prompts tells the LLM to expect the Observation
        # in triple single-quotes.
        return f"""```
Observation: {content}
```
"""
    
    def from_action_response(tool_response: str):
        return ReactObservation.format_observation(
            tool_response
        )
    
    def from_action_error(ar:ReactAssistantResponse, e):
        return ReactObservation.format_observation(
            f"There was an error executing `Action: {ar.action}` with `Action Input: {ar.action_input}. "
            f"The python excepion is as follows: {str(e)}. "
            f"If possible, try again with a different action or different inputs. Remember, pay attention to JSON formatting"
        )

    def from_missing_action(assistant_response:ReactAssistantResponse, tools: ToolCollection):
        return ReactObservation.format_observation(
            f"The Action `{assistant_response.action}` is unknown. Cannot execute it. "
            f"Try one of the available ones [{", ".join(tools.get_tool_names())}]"
        )


#-------------------------------------------------------------------------------
def react_observation_from_action(ar:ReactAssistantResponse, tools : ToolCollection):
    """
    The Observation indicates continutation. If this function returns None, that means
    the react-lop has ended.
    """
    if ar.answer:
        logging.debug("Terminating ReAct loop as an answer has been provided.")
    else:
        logging.debug("Continuing react loop. Executing Action asked for.")
        assert(ar.action)

        if tools.has_tool(ar.action):
            try:
                logging.debug(f"Executing action/tool {ar.action} with args {ar.action_input}")
                action_response = tools.exec_tool(
                    name = ar.action, 
                    args = ar.action_input)
                
                return ReactObservation.from_action_response(str(action_response))
            
            except Exception as e:
                logging.error(f"Executing action raise {str(e)}")
                return ReactObservation.from_action_error(ar, e)
        else:
            logging.error(f"Assistant asked for Action:{ar.action}. There is no such tool!")
            return ReactObservation.from_missing_action(ar)

In [13]:

# Start with the tool_calling agent's run_chat_loop method.
def run_react_loop(sys_prompt: str, start_prompt:str, tools : ToolCollection | None):
    """
    Runs a chat loop with an initial prompt and supplied tools
    Resolves all tool_calls made till a final assistant response is provided

    If a tool_call is made by the LLM and no tools are supplied, a ValueError is raised.
    """
    if not sys_prompt  : raise ValueError("run_react_loop: `sys_prompt` must be supplied")
    if not start_prompt: raise ValueError("run_react_loop: `start_prompt` must be supplied")

    # Initialize
    chat_history = [ { "role" : "system", "content" : sys_prompt}]    

    # Not sure if we should be supplying tools via `tools=` or only 
    # executing the ones that are embedded in the sys_prompt ?
    # For now, null this out.
    tool_schemas = [] # tools.get_schemas() if tools else []

    # Run the loop
    # The msgs list also controls loop continutation. When msgs is empty, 
    # the loop ends
    msgs = [{
        "role":"user", 
        "content": start_prompt}]
    
    while len(msgs):
        chat_history.extend(msgs)
        msgs = []

        response = oai.get_response(
            chat_history=chat_history,
            tools = tool_schemas)

        # tool-call
        # Note: The OpenAI example is outdated
        # tool_calls is not longer a JSON object but an array of 
        # `ChatCompletionMessageToolCall` objects
        if response.choices[0].message.tool_calls:

            # We do not expect actual tool_calls
            # These comes in indirectly via an assistant response that 
            # asks for an 'Action`.
            # We could later extend this into either
            #   - An observation that it is making a tool-call instead of 
            #     sending an action. Turn this exception into an Observation.
            raise NotImplementedError("Got tool-call in react-loop. Not implemented")            
        else:
            # Assistant response
            chat_response = response.choices[0].message.content       
            DH.text(
                "<br>".join(f"Assistant: {chat_response}".split('\n')),
                fg="green")            
            logging.info(f"Received assistant response : {chat_response}")

            # Add response to chat response
            chat_history.append({
                "role" : "assistant",
                "content" : chat_response
            })

            # Followup on the assistant response
            parsed_response = ReactAssistantResponse(chat_response)
            print(str(parsed_response))

            match react_observation_from_action(parsed_response, tools):
                case None:
                    # Nothing added to msgs.
                    # loop will end.
                    logging.debug("🛑 React loop is terminated")
                case o:                    
                    DH.text(o, fg="black", bg="orange")
                    msgs.append({
                        "role" : "user",
                        "content": o
                    })

    
    # return final chat_history item as the response.
    # assert that it is from assistant ?
    return chat_history[-1]["content"]

In [14]:
# What is 20+(2*4)? Calculate step by step
sys_prompt     = builder.build_safe()
react_question = "What is 20+(2*4)? Calculate step by step"

run_react_loop(sys_prompt=sys_prompt, 
               start_prompt=react_question, 
               tools=react_1_tc)

Thought     : I need to calculate the expression step by step, starting with the multiplication.
Action      : multiply
Action Input: {"a": 2, "b": 4}


Thought     : Now that I have the result of the multiplication (2 * 4 = 8), I can proceed to add it to 20.
Action      : add
Action Input: {"a": 20, "b": 8}


Thought     : I can answer without using any more tools.
Action      : None
Action Input: None


'```\nThought: I can answer without using any more tools.\nAnswer: The result of the expression 20 + (2 * 4) is 28.\n```'

# Scratchpad

Snippets while building the loops

## Develop the react loop

 - Start with the tool_calling agent's run_chat_loop method.
 - Just send prompt through and see if&how the assistant response comes through
 - Create new `ReactResponse` class to parse the assistant response 
 - Incorporate the `ReactResponse` into the react loop
   - new `react_observation_from_action`

In [15]:
# 👉 Run the Cell above to call run_react_loop() once you execute this cell
# that will make it pickup the ReactObservation and run_react_loop from this cell
import py_llm.llm.openai_util as oai
from   py_llm.llm.tools import ToolCollection

class ReactObservation:    
    def format_observation(content:str):
        # Note that our system prompts tells the LLM to expect the Observation
        # in triple single-quotes.
        return f"""```
Observation: {content}
```
"""
    
    def from_action_response(tool_response: str):
        return ReactObservation.format_observation(
            tool_response
        )
    
    def from_action_error(ar:ReactAssistantResponse, e):
        return ReactObservation.format_observation(
            f"There was an error executing `Action: {ar.action}` with `Action Input: {ar.action_input}. "
            f"The python excepion is as follows: {str(e)}. "
            f"If possible, try again with a different action or different inputs. Remember, pay attention to JSON formatting"
        )

    def from_missing_action(assistant_response:ReactAssistantResponse, tools: ToolCollection):
        return ReactObservation.format_observation(
            f"The Action `{assistant_response.action}` is unknown. Cannot execute it. "
            f"Try one of the available ones [{", ".join(tools.get_tool_names())}]"
        )


#-------------------------------------------------------------------------------
def react_observation_from_action(ar:ReactAssistantResponse, tools : ToolCollection):
    """
    The Observation indicates continutation. If this function returns None, that means
    the react-lop has ended.
    """
    if ar.answer:
        logging.debug("Terminating ReAct loop as an answer has been provided.")
    else:
        logging.debug("Continuing react loop. Executing Action asked for.")
        assert(ar.action)

        if tools.has_tool(ar.action):
            try:
                logging.debug(f"Executing action/tool {ar.action} with args {ar.action_input}")
                action_response = tools.exec_tool(
                    name = ar.action, 
                    args = ar.action_input)
                
                return ReactObservation.from_action_response(str(action_response))
            
            except Exception as e:
                logging.error(f"Executing action raise {str(e)}")
                return ReactObservation.from_action_error(ar, e)
        else:
            logging.error(f"Assistant asked for Action:{ar.action}. There is no such tool!")
            return ReactObservation.from_missing_action(ar)

#-------------------------------------------------------------------------------
def run_react_loop(sys_prompt: str, start_prompt:str, tools : ToolCollection | None):
    """
    Runs a chat loop with an initial prompt and supplied tools
    Resolves all tool_calls made till a final assistant response is provided

    If a tool_call is made by the LLM and no tools are supplied, a ValueError is raised.
    """
    if not sys_prompt  : raise ValueError("run_react_loop: `sys_prompt` must be supplied")
    if not start_prompt: raise ValueError("run_react_loop: `start_prompt` must be supplied")

    # Initialize
    chat_history = [ { "role" : "system", "content" : sys_prompt}]    

    # Not sure if we should be supplying tools via `tools=` or only 
    # executing the ones that are embedded in the sys_prompt ?
    # For now, null this out.
    tool_schemas = [] # tools.get_schemas() if tools else []

    # Run the loop
    # The msgs list also controls loop continutation. When msgs is empty, 
    # the loop ends
    msgs = [{
        "role":"user", 
        "content": start_prompt}]
    
    while len(msgs):
        chat_history.extend(msgs)
        msgs = []

        response = oai.get_response(
            chat_history=chat_history,
            tools = tool_schemas)

        # tool-call
        # Note: The OpenAI example is outdated
        # tool_calls is not longer a JSON object but an array of 
        # `ChatCompletionMessageToolCall` objects
        if response.choices[0].message.tool_calls:

            # We do not expect actual tool_calls
            # These comes in indirectly via an assistant response that 
            # asks for an 'Action`.
            # We could later extend this into either
            #   - An observation that it is making a tool-call instead of 
            #     sending an action. Turn this exception into an Observation.
            raise NotImplementedError("Got tool-call in react-loop. Not implemented")            
        else:
            # Assistant response
            chat_response = response.choices[0].message.content       
            jh.text(
                "<br>".join(f"Assistant: {chat_response}".split('\n')),
                fg="green")            
            logging.info(f"Received assistant response : {chat_response}")

            # Add response to chat response
            chat_history.append({
                "role" : "assistant",
                "content" : chat_response
            })

            # Followup on the assistant response
            parsed_response = ReactAssistantResponse(chat_response)
            print(str(parsed_response))

            match react_observation_from_action(parsed_response, tools):
                case None:
                    # Nothing added to msgs.
                    # loop will end.
                    logging.debug("🛑 React loop is terminated")
                case o:                    
                    jh.text(o, fg="pink", bg="yellow")
                    msgs.append({
                        "role" : "user",
                        "content": o
                    })

    
    # return final chat_history item as the response.
    # assert that it is from assistant ?
    return chat_history[-1]["content"]

## Parse Assistant responses into out react semantics

 - create new as `ReactResponse`
 - rename to `ReactAssistantResponse`

It sent the following but wrapped in triple quotes. I can't show those quotes in markdown since it is markdown code-block start/end.

```
Thought: I need to calculate the expression step by step.
Action: functions.multiply
Action Input: {"a": 2, "b": 4}
```

 - ❓`functions.` where is this coming from ? I can add it to the regex for sure, but it is unexpected
 - the leading triple-quotes. Was that taken from the exemplars in the system prompt ?

In [16]:
# First assistant response to "What is 20+(2*4)? Calculate step by step"
resp = """```
Thought: I need to calculate the expression step by step.
Action: functions.multiply
Action Input: {"a": 2, "b": 4}
```""".strip()

import re

pat_th = re.compile(r"^(Thought|Action|Action Input)\s*:\s*(.*?)$", re.MULTILINE)
match_list = pat_th.findall(resp)#"Thought: I need to calculate the expression step by step.")
if match_list or len(match_list) > 0:    
    print(f"Have {len(match_list)} matches")
    for m in match_list:    
        print(f"Key = {m[0]}")
        print(f"Value = {m[1]}")
else:
    print(f"Not a valid (Thought/Action/Action Input) response")

DM.hr()

pat_act = re.compile(r"^\s*(?:function[s]?\.)?(.*)$")
m_act   = pat_act.match(" functions.multiply")
if m_act:
    print(m_act.group(1))


Have 3 matches
Key = Thought
Value = I need to calculate the expression step by step.
Key = Action
Value = functions.multiply
Key = Action Input
Value = {"a": 2, "b": 4}


----

multiply


In [28]:
import re
import logging

class ReactAssistantResponse:
    PATTERN_TH        = re.compile(r"^(Thought|Action|Action Input|Answer)\s*:\s*(.*?)$", re.MULTILINE)
    PATTERN_FUNC_NAME = re.compile(r"^\s*(?:function[s]?\.)?(.*)$")

    def __init__(self, assistant_response:str):
        self.thought      = None
        self.action       = None
        self.action_input = None
        self.answer       = None

        #-- Parse -----------------        
        match_list = self.PATTERN_TH.findall(assistant_response)

        if match_list or len(match_list) > 0:
            d = {}
            for m in match_list:
                key = m[0]
                val = m[1]
                logging.debug(f"Extracted [{key} = {val}] pair")
                d[key] = val
            self._init_kvps(d)
        else:
            logging.debug("Could not extract any exected React semantic sections from assistant response\n{assistant_response}")
    
    def __str__(self):
        return f"""
Thought     : {self.thought}
Action      : {self.action}
Action Input: {self.action_input}""".strip()

    def _init_kvps(self, d):
        for k,v in d.items():
            match k.lower():
                case "thought":
                    self.thought = v.strip()
                case "action":
                    # These seem to sometimes comes in as `function.my_action`
                    fn_match = self.PATTERN_FUNC_NAME.match(v)
                    if fn_match:
                        self.action = fn_match.group(1)
                        logging.debug(f"Got Action = {self.action}")
                    else:
                        logging.error(f"Unable to get function name from Action=\"{v}\"")
                case "action input":
                    self.action_input = v
                    logging.debug(f"Got Action Input = {v}")
                case "answer":
                    self.answer = v
                    logging.debug(f"Got Answer = {v}")
                case _:
                    logging.warning(f"Unknown Key={k} with Value={v}")

#-- test -----------------------
# # First assistant response to "What is 20+(2*4)? Calculate step by step"
resp = """```
Thought: I need to calculate the expression step by step.
Action: functions.multiply
Action Input: {"a": 2, "b": 4}
```""".strip()

rr = ReactAssistantResponse(resp)
print(str(rr))

02:34:13 DEBUG:Extracted [Thought = I need to calculate the expression step by step.] pair
02:34:13 DEBUG:Extracted [Action = functions.multiply] pair
02:34:13 DEBUG:Extracted [Action Input = {"a": 2, "b": 4}] pair
02:34:13 DEBUG:Got Action = multiply
02:34:13 DEBUG:Got Action Input = {"a": 2, "b": 4}


Thought     : I need to calculate the expression step by step.
Action      : multiply
Action Input: {"a": 2, "b": 4}
